# Portfolio Optimization: 130/30 Strategy
Ledoit-Wolf shrinkage with constrained optimization

In [ ]:
import pandas as pd
import numpy as np
import cvxpy as cp
from plotnine import *
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load returns from CSV (format: date, asset1, asset2, ...)
returns = (pd.read_csv('returns_data.csv')
           .assign(date=lambda x: pd.to_datetime(x.date))
           .set_index('date')
           .dropna())

print(f"Data: {returns.index[0]:%Y-%m} to {returns.index[-1]:%Y-%m}")
print(f"Assets: {returns.shape[1]}, Periods: {returns.shape[0]}")
returns.head()

In [ ]:
def calc_lw_cov(X):
    """Ledoit-Wolf covariance shrinkage"""
    X = X.values
    n, p = X.shape
    X_centered = X - X.mean(axis=0)
    S = np.cov(X, rowvar=False)
    
    beta_sum = sum(np.sum((np.outer(row, row) - S)**2) for row in X_centered)
    beta_sq = beta_sum / n**2
    
    mu = np.mean(np.diag(S))
    F = np.eye(p) * mu
    delta_sq = np.sum((S - F)**2)
    lambda_shrink = min(1, beta_sq / delta_sq)
    
    return lambda_shrink * F + (1 - lambda_shrink) * S

def optimize_130_30(cov_matrix, mu=None, portfolio_type="gmv", rf=0.035/12):
    """130/30 portfolio optimization"""
    p = cov_matrix.shape[0]
    w_long, w_short = cp.Variable(p, nonneg=True), cp.Variable(p, nonneg=True)
    w = w_long - w_short
    
    constraints = [cp.sum(w_long) == 1.3, cp.sum(w_short) == 0.3]
    
    if portfolio_type == "gmv":
        objective = cp.Minimize(cp.quad_form(w, cov_matrix))
    else:
        excess_ret = np.array(mu) - rf
        objective = cp.Maximize(excess_ret.T @ w - 0.001 * cp.quad_form(w, cov_matrix))
    
    cp.Problem(objective, constraints).solve(verbose=False)
    return w.value

In [ ]:
# Rolling backtest
results = []
window = 60

for i in range(window, len(returns)):
    train = returns.iloc[i-window:i]
    mu = train.mean().values
    cov = calc_lw_cov(train)
    
    gmv_w = optimize_130_30(cov, portfolio_type="gmv")
    orp_w = optimize_130_30(cov, mu, "orp")
    
    if i < len(returns) - 1:
        next_ret = returns.iloc[i+1].values
        results.append({
            'date': returns.index[i],
            'gmv_return': np.dot(gmv_w, next_ret),
            'orp_return': np.dot(orp_w, next_ret),
            'gmv_weights': gmv_w,
            'orp_weights': orp_w
        })

backtest = pd.DataFrame(results)
print(f"Backtest periods: {len(backtest)}")

In [ ]:
# Performance analysis
perf = (backtest
        .assign(gmv_cumret=lambda x: (1 + x.gmv_return).cumprod(),
                orp_cumret=lambda x: (1 + x.orp_return).cumprod()))

metrics = []
for port in ['gmv', 'orp']:
    ret_col = f'{port}_return'
    cum_col = f'{port}_cumret'
    
    total_ret = perf[cum_col].iloc[-1] - 1
    ann_ret = perf[cum_col].iloc[-1] ** (12/len(perf)) - 1
    vol = perf[ret_col].std() * np.sqrt(12)
    sharpe = (perf[ret_col].mean() * np.sqrt(12)) / vol
    
    metrics.append({
        'portfolio': port.upper(),
        'total_return': f"{total_ret:.1%}",
        'ann_return': f"{ann_ret:.1%}",
        'volatility': f"{vol:.1%}",
        'sharpe': f"{sharpe:.2f}"
    })

pd.DataFrame(metrics)

In [ ]:
# Exposure verification
weights_long = []
for _, row in backtest.iterrows():
    for port, weights in [('gmv', row.gmv_weights), ('orp', row.orp_weights)]:
        long_exp = np.sum(np.maximum(weights, 0))
        short_exp = np.sum(np.maximum(-weights, 0))
        weights_long.append({
            'date': row.date, 'portfolio': port,
            'long': long_exp, 'short': short_exp,
            'net': np.sum(weights), 'gross': np.sum(np.abs(weights))
        })

exposure_df = pd.DataFrame(weights_long)
print("Average Exposures (Target: 130% long, 30% short):")
exposure_df.groupby('portfolio')[['long', 'short', 'net', 'gross']].mean().round(3)

In [ ]:
# Cumulative returns plot
(perf
 .melt(id_vars='date', value_vars=['gmv_cumret', 'orp_cumret'],
       var_name='portfolio', value_name='cumret')
 .pipe(lambda x: 
    ggplot(x, aes('date', 'cumret', color='portfolio')) +
    geom_line(size=1.2) +
    labs(title="Cumulative Returns: 130/30 Strategy",
         x="Date", y="Cumulative Return") +
    theme_minimal() +
    scale_color_discrete(name="Portfolio",
                        labels=["GMV", "ORP"])))

In [ ]:
# Exposure plot
(exposure_df
 .query("portfolio == 'gmv'")
 .melt(id_vars='date', value_vars=['long', 'short'])
 .pipe(lambda x:
    ggplot(x, aes('date', 'value', color='variable')) +
    geom_line(size=1) +
    geom_hline(yintercept=1.3, linetype='dashed', alpha=0.5) +
    geom_hline(yintercept=0.3, linetype='dashed', alpha=0.5) +
    labs(title="Portfolio Exposures (GMV)",
         x="Date", y="Exposure") +
    theme_minimal()))

In [ ]:
# Export results
perf.to_csv("portfolio_returns.csv", index=False)
exposure_df.to_csv("exposure_analysis.csv", index=False)
print("Results exported to CSV files")
print(f"Final GMV Sharpe: {(perf.gmv_return.mean() * np.sqrt(12)) / (perf.gmv_return.std() * np.sqrt(12)):.2f}")
print(f"Final ORP Sharpe: {(perf.orp_return.mean() * np.sqrt(12)) / (perf.orp_return.std() * np.sqrt(12)):.2f}")